# Transformer Sentence Embedding

In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
torch.set_printoptions(sci_mode=False)

In [2]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

text = "today I solved some excellent algorithmic problems"

## List All Tokens

In [3]:
encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

tokens = encoded.input_ids.tolist()[0]

for i, token_id in enumerate(tokens):
    decoded_token = tokenizer.decode(token_id)
    print(f"Token {i + 1}: {decoded_token}")

Token 1: [CLS]
Token 2: today
Token 3: i
Token 4: solved
Token 5: some
Token 6: excellent
Token 7: algorithm
Token 8: ##ic
Token 9: problems
Token 10: [SEP]

## Token IDs

In [4]:
print(encoded['input_ids'])
print(encoded['attention_mask'])

tensor([[  101,  2651,  1045, 13332,  2070,  6581,  9896,  2594,  3471,   102]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

## Initial Token Embeddings (Pre-Transformer)

In [5]:
initial_embeddings = model.embeddings(encoded.input_ids)
print(initial_embeddings.shape)
print(initial_embeddings)

torch.Size([1, 10, 384])
tensor([[[-0.1766, -0.0482,  0.0377,  ...,  0.0310,  0.1154, -0.2001],
         [ 0.1899,  0.4778, -0.6466,  ...,  0.0499, -0.5361,  0.4469],
         [-0.2715, -0.5152,  0.1800,  ..., -0.5518, -0.7233, -0.7180],
         ...,
         [-0.7215, -0.0884,  0.2669,  ...,  0.9165, -0.4877, -0.6912],
         [-0.5559, -0.1996, -0.2737,  ...,  0.8535, -0.2942,  0.7360],
         [ 0.2826,  0.1163, -0.2290,  ...,  0.1418, -0.0491, -0.1000]]],
       grad_fn=<NativeLayerNormBackward0>)

## Processing Embeddings with the Transformer

In [6]:
output = model(**encoded)
print(output.last_hidden_state.shape)
print(output.last_hidden_state)

torch.Size([1, 10, 384])
tensor([[[-0.1642,  0.4660,  0.0489,  ...,  0.2386, -0.0663, -0.0930],
         [-0.5260,  0.8277,  0.7240,  ..., -0.7451, -1.1621,  1.1199],
         [ 0.1473,  0.2505,  0.1579,  ..., -0.2087, -1.1386, -0.6368],
         ...,
         [-0.5978,  0.6489, -0.2759,  ...,  0.2102, -0.1217, -0.2887],
         [-0.8411,  0.7750,  0.2105,  ...,  1.0205, -0.1044,  0.5411],
         [-0.3763,  0.5162, -0.3255,  ...,  0.2374, -0.0547, -0.1911]]],
       grad_fn=<NativeLayerNormBackward0>)

## Mean Pooling

Since the attention mask consists entirely of `1`s (no padding tokens),
we can safely compute the simple mean over all token embeddings. If the
attention mask contains `0`s (indicating padding), a weighted mean that
accounts for valid tokens is required instead.

In [7]:
assert torch.all(encoded['attention_mask'] == 1)

In [8]:
pooling = output.last_hidden_state.mean(dim=1)
print(pooling)

tensor([[    -0.4843,      0.5799,      0.1977,     -0.4021,     -0.1579,
             -0.6034,      0.2125,     -0.1393,     -0.8927,      0.0554,
             -0.2813,      0.2256,      0.2001,      0.6244,     -0.1812,
              0.4415,     -0.5326,      0.3251,     -0.2603,     -0.4146,
             -0.8319,      0.1059,     -0.3719,      0.0210,     -0.0178,
              0.4471,      0.1659,     -0.6292,      0.3771,     -0.2244,
             -0.1410,      0.2364,      0.2751,      0.0318,     -0.1013,
              0.1692,     -0.1336,     -0.0603,     -0.1009,      0.1103,
             -0.1001,     -0.0398,      0.0316,     -0.1058,      0.1335,
             -0.0435,     -0.2680,      0.2892,      0.2959,     -0.4699,
             -0.6845,     -0.3565,     -0.4861,     -0.3331,     -0.1521,
             -0.5225,      0.3661,     -0.0263,      0.2539,     -0.3906,
              0.2157,     -0.1258,     -0.2334,      0.0285,      0.3230,
              0.2593,      0.0283,    